In [1]:
#Importando bibliotecas
import pandas as pd         #Usada para a manipulação dos dados como tabela
import re                   #Usada para processar texto
import mysql.connector      #Usada para exportar como SQL
import time                 #Usada apenas para medir a velocidade de execução do código

In [2]:
#Importando arquivo csv
df = pd.read_csv('empresas_IA.csv')

In [3]:
tic=time.time()
def mm(x):
    if len(x.split())==3: return re.sub(r"[^A-Za-z0-9 ]","",x.split()[2])
    else: return 'NaN'
    
def cc_min(x):
    if len(x.split())==5: return re.sub(r"[^A-Za-z0-9 ]","",x.split()[0])
    elif len(x.split())==4: return '0'
    elif len(x.split())==3: return re.sub(r"[^A-Za-z0-9 ]","",x.split()[0])
    else: return x.split()[0]

def cc_max(x):
    if len(x.split())==5: return re.sub(r"[^A-Za-z0-9 ]","",x.split()[2])
    elif len(x.split())==4: return re.sub(r"[^A-Za-z0-9 ]","",x.split()[1])
    elif len(x.split())==3: return 'Undisclosed'
    else: return x.split()[0]

def cidade(x):
    # Cidade é o valor antes da vírgula em 'Localização'
    try: return x.split(',')[0].strip()            
    except: return 'NaN'

def estado(x):
    try:
        # Estado tem formato de duas letras após a vírgula
        if len(x.split(',')[1].strip()) == 2: return x.split(',')[1].strip()
        else: return 'NaN'
    except: return 'NaN'
    
def pais(x):
    try:
        # Se tem estado, país é EUA
        if len(x.split(',')[1].strip()) == 2: return 'EUA'
        # Senão, país é o valor depois da vírgula
        else: return x.split(',')[1].strip()
    except: return 'NaN'

# A partir da coluna "Localização", define 'Cidade', 'Estado' e 'País' usando funções
df['Cidade'] = df['Localização'].map(lambda x: cidade(x))
df['Estado'] = df['Localização'].map(lambda x: estado(x))
df['País'] = df['Localização'].map(lambda x: pais(x))

# Meramente remove o % das células da coluna
df['Porcentagem de serviços de IA']=df['Porcentagem de serviços de IA'].map(lambda x: re.sub(r"[^A-Za-z0-9 ]","",x))

# Remove os caracteres especiais das células da coluna
df['Tamanho minimo do projeto']=df['Tamanho minimo do projeto'].map(lambda x: re.sub(r"[^A-Za-z0-9 ]","",x))

# Número mínimo de empregados é sempre o primeiro valor
df['Número mínimo de empregados'] = df['Numero de empregados'].map(lambda x: re.sub(r"[^A-Za-z0-9 ]","",x.split()[0]))
# Para número máximo, encontrar valor com a função 'mm(x)'
df['Número máximo de empregados'] = df['Numero de empregados'].map(lambda x: mm(x))

# Os valores de custos apresentam-se em diferentes formatos, devidamente tratados nas funções
df['Custo mínimo por hora'] = df['Custo médio por hora'].map(lambda x: cc_min(x))
df['Custo máximo por hora'] = df['Custo médio por hora'].map(lambda x: cc_max(x))

# Remove as colunas que não serão mais usadas
df=df.drop(['Custo médio por hora'],axis=1)
df=df.drop(['Numero de empregados'],axis=1)
df=df.drop(['Localização'],axis=1)

# Renomeia a coluna, colocando acento em mÍnimo
df.rename(columns = {'Tamanho minimo do projeto':'Tamanho mínimo do projeto'}, inplace = True)

# Organiza a ordem das colunas
cols = ['Nome','Website','Cidade','Estado','País','Tamanho mínimo do projeto',
        'Custo mínimo por hora','Custo máximo por hora',
        'Número mínimo de empregados','Número máximo de empregados',
       'Porcentagem de serviços de IA']
df = df[cols]

# Gera uma lista de tuplas com os valores das linhas
dados = [tuple(r) for r in df.to_numpy()]

# Conecta ao MySQL
banco = mysql.connector.connect(host='localhost',user='root',password='')
cursor = banco.cursor()
cursor.execute('CREATE DATABASE companies') #Cria o banco de dados 'companies'
banco = mysql.connector.connect(host='localhost',user='root',password='', database='companies')
cursor = banco.cursor()

# Cria a tabela 'companies' no banco de dados
cursor.execute('''CREATE TABLE companies(name NVARCHAR(500), site NVARCHAR(500),
               city NVARCHAR(50), state NVARCHAR(50), country NVARCHAR(50),
               project_min_size NVARCHAR(50), min_cost_hour NVARCHAR(50), max_cost_hour NVARCHAR(50),
               min_employee_size NVARCHAR(50), max_employee_size NVARCHAR(50), AI_service_percent INT)''')
## Uma vez que havia valores 'NaN' e 'Undisclosed' entre as células, o formato das variáveis foi adaptado
## quando julgou-se necessário.

# Insere múltiplos valores simultaneamente (as tuplas em 'dados')
cursor.executemany("""INSERT INTO companies (name, site, city, state, country, project_min_size,
                min_cost_hour, max_cost_hour, min_employee_size, max_employee_size, AI_service_percent)
                VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""", dados)

tac = time.time()
print('Tempo decorrido: ',round(tac-tic,5),' segundo',sep='')

Tempo decorrido: 1.09573 segundo


In [4]:
df

,Nome,Website,Cidade,Estado,País,Tamanho mínimo do projeto,Custo mínimo por hora,Custo máximo por hora,Número mínimo de empregados,Número máximo de empregados,Porcentagem de serviços de IA
0,Smith.ai,https://smith.ai/,Los Altos,CA,EUA,Undisclosed,Undisclosed,Undisclosed,250,999,10
1,MooveIt,https://moove-it.com/,Austin,TX,EUA,100000,50,99,50,249,15
2,TechSpeedInc,http://www.techspeed.com/,Portland,OR,EUA,1000,0,25,50,249,40
3,WillDom,https://willdom.com/,Fort Lauderdale,FL,EUA,10000,50,99,250,999,10
4,Icreon,https://www.icreon.com/en,New York,NY,EUA,50000,100,149,250,999,10
...,...,...,...,...,...,...,...,...,...,...,...
3095,HurekaTechnologiesInc,http://www.hurekatek.com,New Brunswick,NJ,EUA,10000,25,49,10,49,20
3096,MYZEALI.T.SolutionsLLC,http://www.myzealit.com/,Los Angeles,CA,EUA,10000,25,49,250,999,10
3097,Intelia,https://intelia.io/,Lagos,NaN,Nigeria,25000,Undisclosed,Undisclosed,10,49,20
3098,MojoTrek,http://www.mojotrek.com,Chicago,IL,EUA,10000,50,99,10,49,25
